# Python 作用域与作用域链

## 思路简述

Python 作用域决定了变量的可访问范围，作用域链是 Python 查找变量时遵循的 LEGB 规则（局部 -> 嵌套 -> 全局 -> 内置）。

## 关键步骤

- 理解四种作用域：Local（局部）、Enclosing（嵌套）、Global（全局）、Built-in（内置）
- 掌握查找顺序：Python 按照从内到外的顺序查找变量
- 理解闭包机制：嵌套函数可以访问外层函数的变量
- 使用关键字控制：global 和 nonlocal 修改作用域行为

## 核心要点

- Python 查找变量时从最内层开始，逐层向外查找
- global 让函数修改全局变量
- nonlocal 让嵌套函数修改外层函数的变量
- 作用域链保证了变量的封装性和可访问性

## Q & A

In [1]:
"""
题目一：LEGB 查找规则（基础）
"""

x = "A"

def func1():
    x = "B"

    def func2():
        x = "C"
        print(f"func2: {x}")
    
    func2()
    print(f"func1: {x}")

func1()
print(f"global: {x}")

"""
问题：请写出输出结果，并解释每个 print 找到的是哪个作用域的 x。
"""


func2: C
func1: B
global: A


'\n问题：请写出输出结果，并解释每个 print 找到的是哪个作用域的 x。\n'

**解析：每个函数优先查找自己的局部作用域，找不到才向外查找。**

In [2]:
"""
题目二：global 关键字（基础）
"""

count = 0

def increment():
    count += 1 # type: ignore
    return count

def increment_with_global():
    global count
    count += 1
    return count

print(increment())
print(increment_with_global())
print(increment())

"""
问题：哪一行报错？为什么？写出正确的输出结果。
"""


UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

**解析：第一个函数中 `count += 1` 被视为创建局部变量，但赋值前就引用了，报错。`global` 关键字让第二个函数正确修改全局变量。**

In [3]:
"""
题目三：nonlocal 关键字（进阶）
"""

def outer():
    value = 10

    def inner():
        value = 20
        print(f"inner: {value}")
    
    inner()
    print(f"outer: {value}")

def outer_with_nonlocal():
    value = 10

    def inner():
        nonlocal value
        value = 20
        print(f"inner: {value}")
    
    inner()
    print(f"outer: {value}")

outer()
print("-----")
outer_with_nonlocal()

"""
问题：两个函数的输出有什么不同？nonlocal 在这里起了什么作用？
"""

inner: 20
outer: 10
-----
inner: 20
outer: 20


'\n问题：两个函数的输出有什么不同？nonlocal 在这里起了什么作用？\n'

**解析：没有 `nonlocal` 时，`inner` 创建新的局部变量；有 `nonlocal` 时，`inner` 修改外层函数的变量。**

In [4]:
"""
题目四：闭包中的作用域（进阶）
"""

def make_multiplier(factor):
    def multiply(number):
        return number * factor
    
    return multiply

double = make_multiplier(2)
triple = make_multiplier(3)

print(double(5))
print(triple(5))
print(double(10))

"""
问题：factor 变量在哪个作用域？为什么 double 和 triple 能记住各自的 factor 值？
"""

10
15
20


'\n问题：factor 变量在哪个作用域？为什么 double 和 triple 能记住各自的 factor 值？\n'

**解析：`factor` 在 `make_multiplier` 的作用域中，闭包机制让 `multiply` 函数记住了创建时的 `factor` 值。**

In [5]:
"""
题目五：变量遮蔽（进阶）
"""

x = 100

def shadow_test():
    x = 200
    print(f"局部 x：{x}")

    def nested():
        # x = 300
        print(f"嵌套 x：{x}")
    
    nested()
    print(f"回到局部 x：{x}")

shadow_test()
print(f"全局 x：{x}")

"""
问题：如果取消注释 x = 300，输出会如何变化？这说明了什么？
"""

局部 x：200
嵌套 x：200
回到局部 x：200
全局 x：100


'\n问题：如果取消注释 x = 300，输出会如何变化？这说明了什么？\n'

**解析：内层作用域的变量会遮蔽外层同名变量，但不会修改外层变量。**

In [6]:
"""
题目六：综合挑战（困难）
"""

result = []

def closure_test():
    count = 0

    for i in range(3):
        def func():
            nonlocal count
            count += 1
            return count
        result.append(func)

    return result

funcs = closure_test()
print(funcs[0]())
print(funcs[1]())
print(funcs[2]())

"""
问题：输出结果是什么？为什么三个函数共享一个 count？如果去掉 nonlocal count 会发生什么？
"""

1
2
3


'\n问题：输出结果是什么？为什么三个函数共享一个 count？如果去掉 nonlocal count 会发生什么？\n'

**解析：三个函数共享同一个 `count` 变量（闭包）。去掉 `nonlocal` 会报错，因为 `count += 1` 会尝试创建局部变量。**